In [1]:
from tracker import Tracker
import cv2 as cv
from optical_flow.bounding_box import BoundingBox
from feature_detection.shi_tomasi import ShiTomasiDetector
from optical_flow.lucas_kanade import LucasKanade
from feature_description.sift_descriptor import SIFTDescriptor
from segmentation.fuzzy_c_means import FuzzyCMeansSegmenter
from tqdm import tqdm

In [2]:
video_path = "../videos/jog.mp4"
out_path = "/workspaces/python-opencv/repo/out/alltogether.mp4"

In [3]:
# x, y, w, h = 625, 375, 75, 125
# x, y, w, h = 645, 385, 40, 100
# x, y, w, h = 300, 180, 100, 60
x, y, w, h = 645//2, 385//2, 40//2, 100//2
init_bbox = BoundingBox(x, y, w, h)
target_size = (360, 640)[::-1]

In [4]:
cap = cv.VideoCapture(video_path)
fps = cap.get(cv.CAP_PROP_FPS)

In [5]:
ret, frame = cap.read()
frame = cv.resize(frame, target_size)

In [6]:
# sift_params = {"sigma":5, "nOctaveLayers":3, "contrastThreshold":0.03, "edgeThreshold":10}
sift_params = {"sigma": 4, "nOctaveLayers":3, "contrastThreshold":0.03, "edgeThreshold":10}
# sift_params = {}

detector = ShiTomasiDetector()
optical_flow = LucasKanade()
descriptor = SIFTDescriptor(params=sift_params)
segmenter = FuzzyCMeansSegmenter(5, 0, 10, 2, 2, 10)

In [7]:
tracker = Tracker(frame, init_bbox, detector, optical_flow, descriptor, segmenter)

In [8]:
fourcc = cv.VideoWriter_fourcc(*'mp4v')
video_out = cv.VideoWriter(out_path, fourcc, 30, frame.shape[0:2][::-1]) 

In [ ]:
img2 = cv.rectangle(frame, (init_bbox.x, init_bbox.y), (init_bbox.x+init_bbox.w, init_bbox.y+init_bbox.h), 255, 2)
video_out.write(img2)

ret = True
for i in tqdm(range(150)):

    ret, frame = cap.read()

    if not ret:
        break

    frame = cv.resize(frame, target_size)

    bbox = tracker.track(frame)

    img2 = cv.rectangle(frame, (bbox.x, bbox.y), (bbox.x+bbox.w, bbox.y+bbox.h), 255, 2)

    video_out.write(img2)

In [10]:
cap.release()
video_out.release()